In [ ]:
!pip install -q efficientnet
import tensorflow as tf
import efficientnet.tfkeras as efn
import matplotlib.pyplot as plt
import glob,os
import numpy as np
import pandas as pd
from kaggle_datasets import KaggleDatasets
keras = tf.keras
layers = keras.layers
EPOCH = 20
DATASET_SIZE = 16465
TRAIN_SIZE = 12753
VAL_SIZE = 3712
TEST_SIZE = 7382
LR_START = 1e-3
AUTO = tf.data.experimental.AUTOTUNE
ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU',tpu.master())
except ValueError:
    tpu = None
    
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

BATCH_SIZE = 16 * strategy.num_replicas_in_sync
print(f'tensorflow version : {tf.__version__}')
print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
#dataset_filenames = tf.io.gfile.glob(r'gs://kds-b2e6cdbc4af76dcf0363776c09c12fe46872cab211d1de9f60ec7aec/tfrecords-jpeg-512x512/*a*/*.tfrec')
train_filenames = tf.io.gfile.glob(r'gs://kds-b2e6cdbc4af76dcf0363776c09c12fe46872cab211d1de9f60ec7aec/tfrecords-jpeg-512x512/train/*.tfrec')
val_filenames = tf.io.gfile.glob(r'gs://kds-b2e6cdbc4af76dcf0363776c09c12fe46872cab211d1de9f60ec7aec/tfrecords-jpeg-512x512/val/*.tfrec')
test_filenames = tf.io.gfile.glob(r'gs://kds-b2e6cdbc4af76dcf0363776c09c12fe46872cab211d1de9f60ec7aec/tfrecords-jpeg-512x512/test/*.tfrec')
#count_dataset = np.sum([int(path.split('-')[-1].split('.')[0]) for path in dataset_filenames])
#count_train = np.sum([int(path.split('-')[-1].split('.')[0]) for path in train_filenames])
#count_val = np.sum([int(path.split('-')[-1].split('.')[0]) for path in val_filenames])
#print(count_dataset)
#print(count_train)
#print(count_val)
#dataset = tf.data.TFRecordDataset(dataset_filenames)
#dataset = dataset.with_options(ignore_order)
dataset_train = tf.data.TFRecordDataset(train_filenames)
dataset_train = dataset_train.with_options(ignore_order)
dataset_val = tf.data.TFRecordDataset(val_filenames)
dataset_val = dataset_val.with_options(ignore_order)
dataset_test = tf.data.TFRecordDataset(test_filenames)
dataset_test = dataset_test.with_options(ignore_order)

In [ ]:
feature_description = {
    'class':tf.io.FixedLenFeature([],tf.int64),
    'image':tf.io.FixedLenFeature([],tf.string)
}
test_feature_description = {
    'id':tf.io.FixedLenFeature([],tf.string),
    'image':tf.io.FixedLenFeature([],tf.string)
}
def dataset_decode(data):
    decode_data = tf.io.parse_single_example(data,feature_description)
    label = decode_data['class']
    image = tf.image.decode_jpeg(decode_data['image'],channels=3)
    image = tf.reshape(image,[512,512,3])
    image = tf.image.random_flip_left_right(image)
    image = tf.cast(image,tf.float32)
    image = (image - 127.5) / 127.5
    return image,label
def test_dataset_decode(data):
    decode_data = tf.io.parse_single_example(data,test_feature_description)
    ID = decode_data['id']
    image = tf.image.decode_jpeg(decode_data['image'],channels=3)
    image = tf.reshape(image,[512,512,3])
    image = tf.cast(image,tf.float32)
    image = (image - 127.5) / 127.5
    return ID,image

In [ ]:
#dataset = dataset.map(dataset_decode)
dataset_train = dataset_train.map(dataset_decode)
dataset_val = dataset_val.map(dataset_decode)
dataset_test = dataset_test.map(test_dataset_decode)
#dataset = dataset.shuffle(DATASET_SIZE).repeat().batch(BATCH_SIZE).prefetch(AUTO)
dataset_train = dataset_train.shuffle(DATASET_SIZE).repeat().batch(BATCH_SIZE).prefetch(AUTO)
dataset_val = dataset_val.batch(BATCH_SIZE).prefetch(AUTO)
dataset_test = dataset_test.batch(BATCH_SIZE).prefetch(AUTO)
#print(dataset)
print(dataset_train)
print(dataset_val)
print(dataset_test)

In [ ]:
optimizer = keras.optimizers.Adam(LR_START)
loss = keras.losses.SparseCategoricalCrossentropy()
metrics = keras.metrics.SparseCategoricalAccuracy()
lr_callback = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.8,patience=3,min_lr=1e-6)
with strategy.scope():
    base_network_1 = efn.EfficientNetB7(include_top=False,input_shape=(512,512,3),weights='imagenet')
    #base_network_2 = keras.applications.InceptionResNetV2(include_top=False,input_shape=[512,512,3],weights='imagenet')
    base_network_3 = keras.applications.DenseNet201(include_top=False,input_shape=[512,512,3],weights='imagenet')
    network_1 = keras.Sequential()
    network_1.add(base_network_1)
    network_1.add(layers.MaxPooling2D())
    network_1.add(layers.Conv2D(2560,3,padding='same'))
    network_1.add(layers.BatchNormalization())
    network_1.add(layers.ReLU())
    network_1.add(layers.GlobalAveragePooling2D())
    network_1.add(layers.Dense(1024))
    network_1.add(layers.BatchNormalization())
    network_1.add(layers.LeakyReLU())
    network_1.add(layers.Dense(512))
    network_1.add(layers.BatchNormalization())
    network_1.add(layers.LeakyReLU())
    network_1.add(layers.Dense(104,activation='softmax'))
    network_1.compile(optimizer=optimizer,loss=loss,metrics=[metrics])
    """
    network_2 = keras.Sequential()
    network_2.add(base_network_2)
    network_2.add(layers.MaxPooling2D())
    network_2.add(layers.Conv2D(2560,3,padding='same'))
    network_2.add(layers.BatchNormalization())
    network_2.add(layers.ReLU())
    network_2.add(layers.GlobalAveragePooling2D())
    network_2.add(layers.Dense(1024))
    network_2.add(layers.BatchNormalization())
    network_2.add(layers.LeakyReLU())
    network_2.add(layers.Dense(512))
    network_2.add(layers.BatchNormalization())
    network_2.add(layers.LeakyReLU())
    network_2.add(layers.Dense(104,activation='softmax'))
    network_2.compile(optimizer=optimizer,loss=loss,metrics=[metrics])
    """
    network_3 = keras.Sequential()
    network_3.add(base_network_3)
    network_3.add(layers.MaxPooling2D())
    network_3.add(layers.Conv2D(2560,3,padding='same'))
    network_3.add(layers.BatchNormalization())
    network_3.add(layers.ReLU())
    network_3.add(layers.GlobalAveragePooling2D())
    network_3.add(layers.Dense(1024))
    network_3.add(layers.BatchNormalization())
    network_3.add(layers.LeakyReLU())
    network_3.add(layers.Dense(512))
    network_3.add(layers.BatchNormalization())
    network_3.add(layers.LeakyReLU())
    network_3.add(layers.Dense(104,activation='softmax'))
    network_3.compile(optimizer=optimizer,loss=loss,metrics=[metrics])
network_1.summary()
network_1.fit(dataset_train,
              epochs=EPOCH,
              steps_per_epoch=TRAIN_SIZE//BATCH_SIZE,
              validation_data=dataset_val,
              validation_steps=VAL_SIZE//BATCH_SIZE,
              callbacks=[lr_callback])
"""
network_2.summary()
network_2.fit(dataset_train,
              epochs=EPOCH,
              steps_per_epoch=TRAIN_SIZE//BATCH_SIZE,
              validation_data=dataset_val,
              validation_steps=VAL_SIZE//BATCH_SIZE,
              callbacks=[lr_callback])
"""
network_3.summary()
network_3.fit(dataset_train,
              epochs=EPOCH,
              steps_per_epoch=TRAIN_SIZE//BATCH_SIZE,
              validation_data=dataset_val,
              validation_steps=VAL_SIZE//BATCH_SIZE,
              callbacks=[lr_callback])

In [ ]:
predict_csv = []
for ID,image in dataset_test:
    prediction_1 = network_1.predict(image)
    #prediction_2 = network_2.predict(image)
    prediction_3 = network_3.predict(image)
    prediction = prediction_1 + prediction_3# + prediction_2
    label = tf.argmax(prediction,axis=1).numpy()
    ID = [item.numpy().decode('utf-8') for item in ID]
    for i in range(len(ID)):
        predict_csv.append([ID[i],label[i]])
    print('*',end='')

In [ ]:
csv_name = ['id','label']
csv_data = pd.DataFrame(columns=csv_name,data=predict_csv)
csv_data.to_csv(r'submission.csv',index=False)

In [ ]:
print('DONE')